#### In this section, we are experimenting with OSRM and different routing techniques to create efficient routing algorithm. 

Please make sure the OSRM engine is running in the background with the desired region's map.

#### The most common use of OSRM is to read location coordinate from a CSV file such as locations.csv and create routing between these locations. 

Please see a sample of location.csv for Ethiopia:

name,region,country,latitude,longitude,location_type,conflict_date,population<br>
Addis Ababa,Addis Ababa,Ethiopia,8.978098143949728,38.75857450155794,town,0,0<br>
Mekele,Mekele,Ethiopia,13.495486756898567,39.46589089154361,town,0,0<br>
Dire Dawa,Dire Dawa,Ethiopia,9.602835645348511,41.85443640082678,town,0,0<br>
Adama,Adama,Ethiopia,8.526471189559459,39.25963003366467,town,0,0

#### Please install packages that will be used in the experiment. 

In [ ]:
import sys
!{sys.executable} -m pip install numpy pandas matplotlib geopy folium polyline tsp_solver scikit-learn scipy

#### Please note: In order to avoid code repetition, we have structured the workflow in a way that could introduce dependencies on preceding cells. If you encounter the issue 'function is not defined,' simply run the cell containing the relevant function first. This will ensure that the notebook registers the function and enables other cells to access it.

#### Please run the next two cells to import required libraries and set the experimenting region.

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import requests
import folium
import polyline
from sklearn.cluster import DBSCAN
from scipy.spatial import distance_matrix
from scipy.optimize import minimize
from sklearn.neighbors import KDTree
from tsp_solver.greedy import solve_tsp
import matplotlib.pyplot as plt
from itertools import combinations
from geopy.distance import geodesic

In [ ]:
# Define the region or the map here:

region = "ethiopia"

#### The code below iterates over locations in the regions's locations csv file, makes inquiries to the OSRM engine and costructs simple routing. Please see codes inner comments for more information. 

In [ ]:
# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create an empty dictionary to store distances between pairs
distance_dict = {}

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                    # Append step information to the route_instructions list
                    route_instructions.append((instruction, distance_sum))
                    
                # Store distance between the pair in the dictionary
                distance_dict[(name1, name2)] = round(distance_sum, 2)

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=(lat1, lon1),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=(lat2, lon2),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', f'{region}-route-map-simple.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Save distances to a CSV file
with open(f'{region}-simple-distances.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['name1', 'name2', 'distance'])
    for pair, distance in distance_dict.items():
        writer.writerow([pair[0], pair[1], distance])

print(f"Distances saved as {region}-simple-distances.csv")

# Display the map
m

#### The code below iterates over locations in the regions's locations csv file, makes inquiries to the OSRM engine and costructs routing. It also removes duplicate routings.

In [ ]:
# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty set to store processed route pairs
processed_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the reverse pair already exists in the processed pairs set
        if (name2, name1) in processed_pairs:
            continue

        # Add the pair to the processed pairs set
        processed_pairs.add((name1, name2))

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=(lat1, lon1),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=(lat2, lon2),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', f'{region}-route-map-remove-duplicates.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

#### The code below iterates over locations in the regions's locations csv file, makes inquiries to the OSRM engine and costructs distance-based routing.

In [ ]:
# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty set to store processed route pairs
processed_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Create frozensets for the pair of locations
        location_pair = frozenset([name1, name2])

        # Check if the frozenset already exists in the processed pairs set
        if location_pair in processed_pairs:
            continue

        # Add the frozenset to the processed pairs set
        processed_pairs.add(location_pair)

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Create a Folium PolyLine object
                polyline_obj = folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                )

                # Add the PolyLine to the map
                polyline_obj.add_to(m)

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=(lat1, lon1),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=(lat2, lon2),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', f'{region}-route-map-distance-based.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

#### The code below iterates over locations in the regions's locations csv file, makes inquiries to the OSRM engine and costructs routing through clustering algorithm. Please read inner comments for more information. 

In [ ]:
import os
import pandas as pd
import requests
import folium

# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Create a set to store visited location pairs
visited_pairs = set()

# Iterate through each pair of locations in the DataFrame
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        # Extract location details
        name1 = df.loc[i, 'name']
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        name2 = df.loc[j, 'name']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Check if the pair has been visited before (or its reverse pair)
        if (name1, name2) in visited_pairs or (name2, name1) in visited_pairs:
            continue

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        try:
            # Send request to OSRM
            response = requests.get(request_url)
            response.raise_for_status()

            # Extract route data from the response
            route_data = response.json()

            # Check if response contains a route
            if 'routes' in route_data and len(route_data['routes']) > 0:
                # Extract route steps
                steps = route_data['routes'][0]['legs'][0]['steps']
                distance_sum = 0
                for step in steps:
                    # Extract step instructions
                    instruction = step['name']

                    # Extract step distance and add to the distance sum
                    distance = step['distance']
                    distance_sum += distance

                # Append step information to the route_instructions list
                route_instructions.append((instruction, distance_sum))

                # Extract route geometry
                geometry = route_data['routes'][0]['geometry']

                # Decode polyline string to get coordinates
                coordinates = polyline.decode(geometry)

                # Add the coordinates to the map as a PolyLine
                folium.PolyLine(
                    locations=coordinates,
                    color='blue',
                    weight=2,
                    opacity=0.9
                ).add_to(m)

                # Add the pair to the visited set and its reverse pair
                visited_pairs.add((name1, name2))
                visited_pairs.add((name2, name1))

        except requests.exceptions.RequestException as e:
            print(f"Error occurred during OSRM API request: {e}")
            continue

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=(lat1, lon1),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=(lat2, lon2),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

# Save the map as an HTML file
map_file = os.path.join('images', f'{region}-route-map-clustering.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

#### The code below iterates over locations in the regions's locations csv file, makes inquiries to the OSRM engine and costructs distance-based routing through clustering algorithm.  

In [ ]:
# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
locations_csv_file = os.path.join(locations_csv_path, locations_csv)
if not os.path.exists(locations_csv_file):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(locations_csv_file)

# Create a map centered around the first location
start_lat = df.loc[0, 'latitude']
start_lon = df.loc[0, 'longitude']
m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

# Create an empty list to store route instructions
route_instructions = []

# Iterate through the sequential pairs of locations in the DataFrame
for i in range(len(df)):
    # Extract location details
    name1 = df.loc[i, 'name']
    lat1 = df.loc[i, 'latitude']
    lon1 = df.loc[i, 'longitude']
    
    # Check if it is the last location
    if i == len(df) - 1:
        name2 = df.loc[0, 'name']
        lat2 = df.loc[0, 'latitude']
        lon2 = df.loc[0, 'longitude']
    else:
        name2 = df.loc[i + 1, 'name']
        lat2 = df.loc[i + 1, 'latitude']
        lon2 = df.loc[i + 1, 'longitude']

    # Prepare API request URL
    osrm_endpoint = 'http://localhost:5000/route/v1/driving'
    request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

    try:
        # Send request to OSRM
        response = requests.get(request_url)
        response.raise_for_status()

        # Extract route data from the response
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract route steps
            steps = route_data['routes'][0]['legs'][0]['steps']
            distance_sum = 0
            for step in steps:
                # Extract step instructions
                instruction = step['name']

                # Extract step distance and add to the distance sum
                distance = step['distance']
                distance_sum += distance

            # Append step information to the route_instructions list
            route_instructions.append((instruction, distance_sum))

            # Extract route geometry
            geometry = route_data['routes'][0]['geometry']

            # Decode polyline string to get coordinates
            coordinates = polyline.decode(geometry)

            # Add the coordinates to the map as a PolyLine
            folium.PolyLine(
                locations=coordinates,
                color='blue',
                weight=2,
                opacity=0.9
            ).add_to(m)

        # Add markers for the start and end locations
        folium.CircleMarker(
            location=(lat1, lon1),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name1
        ).add_to(m)

        folium.CircleMarker(
            location=(lat2, lon2),
            radius=6,
            color='red',
            fill=True,
            fill_color='blue',
            popup=name2
        ).add_to(m)

    except requests.exceptions.RequestException as e:
        print(f"Error occurred during OSRM API request: {e}")
        continue

# Save the map as an HTML file
map_file = os.path.join('images', f'{region}-route-map-distance-based-clustering.html')
m.save(map_file)
print(f"Map saved as {map_file}")

# Display the map
m

#### The code below creates and stores Distance Matrix used in the next parts of the routing project.

In [ ]:
# Set the file name of the locations.csv file
locations_csv = f'{region}-locations.csv'

# Set the path to the directory where route.geojson is stored
locations_csv_path = '.'

# Check if the CSV file exists
if not os.path.exists(os.path.join(locations_csv_path, locations_csv)):
    print(f"Error: {locations_csv} file not found.")
    exit(1)

# Read the locations from CSV
df = pd.read_csv(os.path.join(locations_csv_path, locations_csv))

# Create an empty DataFrame to store the distances
num_locations = len(df)
distance_matrix = pd.DataFrame(index=range(num_locations), columns=df['name'])
distance_matrix = distance_matrix.fillna(0)

# Iterate through each pair of locations in the DataFrame
for i in range(num_locations):
    for j in range(i + 1, num_locations):
        # Extract location details
        lat1 = df.loc[i, 'latitude']
        lon1 = df.loc[i, 'longitude']
        lat2 = df.loc[j, 'latitude']
        lon2 = df.loc[j, 'longitude']

        # Prepare API request URL
        osrm_endpoint = 'http://localhost:5000/route/v1/driving'
        request_url = f"{osrm_endpoint}/{lon1},{lat1};{lon2},{lat2}?steps=true&geometries=polyline"

        # Send request to OSRM
        response = requests.get(request_url)
        route_data = response.json()

        # Check if response contains a route
        if 'routes' in route_data and len(route_data['routes']) > 0:
            # Extract distance
            distance = route_data['routes'][0]['distance']

            # Update the distance matrix
            distance_matrix.at[i, df.loc[j, 'name']] = distance
            distance_matrix.at[j, df.loc[i, 'name']] = distance
        else:
            print(f"No route found between {df.loc[i, 'name']} and {df.loc[j, 'name']}")
            # We can choose to set the distance_matrix entries to a specific value
            # to indicate that no route was found, for example:
            # distance_matrix.at[i, df.loc[j, 'name']] = -1
            # distance_matrix.at[j, df.loc[i, 'name']] = -1

# Save the distance matrix to a CSV file for pruning
distance_matrix.to_csv(f'{region}-distance-matrix.csv', index_label='name')

print(f"Distances saved as {region}-distance-matrix.csv")


#### The code below implements Geo Pruning algorithm.

In [ ]:
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters


def geo_pruning(x, df, k, b):
    name = x.name  # Use name instead of index
    distances = []
    x1 = df.loc[name, 'latitude']
    y1 = df.loc[name, 'longitude']
    for name_other in df.index:  # Use name instead of index
        x2 = df.loc[name_other, 'latitude']
        y2 = df.loc[name_other, 'longitude']
        distance = geodesic((x1, y1), (x2, y2)).meters
        distances.append(distance)
        
    if k < len(distances):
        knn = sorted(distances)[k]
    else:
        print(f"Warning: k={k} is out of range for available distances.")
        knn = sorted(distances)[-1]  # Use the maximum distance

    distances = [d if d <= knn * b else 0 for d in distances]
    return distances, name  # Return both distances and name


def get_polyline_string(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/driving/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200 and data.get("code") == "Ok":
        route = data.get("routes")[0]
        polyline_string = route.get("geometry")
        return polyline_string
    else:
        return None
    

def visualise_geo_pruned_routes(locations_df, distances_geo_df, region, k, b):
    # Create a map centered around the first location
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Apply geo_pruning function to get the pruned distances
    geo_pruned_distances = distances_geo_df.apply(lambda x: geo_pruning(x, locations_df, k, b), axis=1)

    # Create routes DataFrame from Pruned Route results
    routes_data = []
    for idx1, row in geo_pruned_distances.items():
        nearest_neighbors = [idx2 for idx2, distance in enumerate(row[0]) if distance > 0]
        for idx2 in nearest_neighbors:
            name1 = locations_df.loc[idx1, 'name']
            name2 = locations_df.loc[idx2, 'name']
            routes_data.append({'location1': name1, 'location2': name2})

    routes_df = pd.DataFrame(routes_data)

    # Iterate through the routes_df DataFrame and update the geometry column
    route_coordinates = []
    for i, row in routes_df.iterrows():
        name1 = row['location1']
        name2 = row['location2']
        idx1 = locations_df[locations_df['name'] == name1].index[0]
        idx2 = locations_df[locations_df['name'] == name2].index[0]
        origin = (locations_df.loc[idx1, 'latitude'], locations_df.loc[idx1, 'longitude'])
        destination = (locations_df.loc[idx2, 'latitude'], locations_df.loc[idx2, 'longitude'])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {idx1} and {idx2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
            route_coordinates.append(coordinates)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for {idx1} to {idx2}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{idx1} to {idx2}"
        )
        polyline_obj.add_to(m)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        if row['location_type'] == 'camp':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='green',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        elif row['location_type'] == 'town':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='darkorange',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        else:
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        
    # Save the route_coordinates to the CSV file
    csv_file = f"{region}-geo-pruned-route-coords.csv"
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["latitude", "longitude"])
        for coordinates in route_coordinates:
            writer.writerows(coordinates)

    print(f"Coordinates saved as {csv_file}")
    
    # Create an empty list to store pruned distances
    pruned_distance_data = []  
    
    # Apply geo_pruning function and get the pruned distances
    for idx, row in distances_geo_df.iterrows():
        distances, idx1 = geo_pruning(row, locations_df, k, b)
        for idx2, distance in enumerate(distances):
            if distance > 0:
                # Map index to location name
                name1 = locations_df.loc[idx1, 'name']
                name2 = locations_df.loc[idx2, 'name']
                # Append to pruned_distance_data up to two decimal places
                pruned_distance_data.append({"location1": name1, "location2": name2, "distance": round(distance, 2)})

    # Convert list of dictionaries to DataFrame
    pruned_distance_df = pd.DataFrame(pruned_distance_data)

    # Save to CSV
    pruned_distance_df.to_csv(f"{region}-geo-pruned-distances.csv", index=False)
    print(f"Pruned distances saved as {region}-geo-pruned-distances.csv")

    # Save the map as an HTML file
    map_file = f'images/{region}-geo-pruned-route-map.html'
    m.save(map_file)
    print(f"Map saved as {map_file}")

    # Display the map
    return m

# Load data here (Locations and Distances DataFrames)
Locations = pd.read_csv(f'{region}-locations.csv')

Distances = pd.read_csv(f'{region}-distance-matrix.csv', index_col='name')

# Set parameters for geo_pruning
k_value = 3
b_value = 1.1

# Visualise geo_pruning results
m_geo_pruned = visualise_geo_pruned_routes(Locations, Distances, region, k_value, b_value)

# Display map
m_geo_pruned

#### The code below implements Triangle Pruning

In [ ]:
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters


def triangle_pruning(x, df, num_neighbors):
    name = x.name
    distances_with_names = []  # Initialize list to store tuples of (name1, name2, distance)
    
    # Extract latitude and longitude of the current location
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    
    # Calculate distances to all other locations
    for idx, row in df.iterrows():
        if row['name'] != name:
            x2 = row['latitude']
            y2 = row['longitude']
            distance = geodesic((x1, y1), (x2, y2)).meters
            distances_with_names.append((name, row['name'], distance))  # Store the names and distance
    
    # Sort distances and select the nearest neighbors
    nearest_neighbors = sorted(distances_with_names, key=lambda x: x[2])[:num_neighbors]
    
    return nearest_neighbors  # Return list of tuples with names and distances


def visualise_triangle_pruned_routes(locations_df, distances_geo_df, region, t):
    # Create a map centered around the first location
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Apply triangle_pruning function to get the pruned distances
    triangle_pruned_distances = distances_geo_df.apply(triangle_pruning, args=(locations_df, t,))
    
    # Create routes DataFrame from Pruned Route results
    routes_data = []
    for idx1, row in triangle_pruned_distances.iterrows():
        nearest_neighbors = row  # row is already the list of nearest neighbor indices
        for idx2 in nearest_neighbors:
            routes_data.append({'idx1': idx1, 'idx2': idx2})
            
    routes_df = pd.DataFrame(routes_data)
    
    # Iterate through the routes_df DataFrame and update the geometry column
    route_coordinates = []
    for i, row in routes_df.iterrows():
        idx1 = row['idx1']  # Use the index as location name
        location_name = row['idx2'][1]  # Extract the second location name from the tuple
        idx2 = locations_df.index[locations_df['name'] == location_name].tolist()[0]  # Convert the name to its corresponding numerical index

        origin = (locations_df.loc[idx1, 'latitude'], locations_df.loc[idx1, 'longitude'])
        destination = (locations_df.loc[idx2, 'latitude'], locations_df.loc[idx2, 'longitude'])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {idx1} and {idx2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
            route_coordinates.append(coordinates)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for {idx1} to {idx2}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{idx1} to {idx2}" 
        )
        polyline_obj.add_to(m)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        if row['location_type'] == 'camp':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='green',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        elif row['location_type'] == 'town':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='darkorange',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        else:
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        
    # Save the route_coordinates to the CSV file
    csv_file = f"{region}-triangle-pruned-route-coords.csv"
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["latitude", "longitude"])
        for coordinates in route_coordinates:
            writer.writerows(coordinates)

    print(f"Coordinates saved as {csv_file}")

    # Save the map as an HTML file
    map_file = f'images/{region}-triangle-pruned-route-map.html'
    m.save(map_file)
    print(f"Map saved as {map_file}")
    
    # Create a CSV file to store location pairs and distances
    csv_file = f"{region}-triangle-pruned-distances.csv"
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["location1", "location2", "distance"])  # Write header

        # Apply triangle_pruning function to get the pruned distances
        triangle_pruned_distances = distances_geo_df.apply(triangle_pruning, args=(locations_df, t,))

        # Write distances to CSV file
        for idx, row in triangle_pruned_distances.iterrows():
            location1 = locations_df.loc[idx, 'name']
            for neighbor in row:
                location2 = neighbor[0]
                distance = neighbor[2]

                # Set distance to zero if location1 and location2 are the same
                if location1 == location2:
                    continue

                # Round distance to two decimal places
                distance = round(distance, 2)

                writer.writerow([location1, location2, distance])  # location1, location2, distance

    print(f"Distances saved as {csv_file}")

    # Display the map
    return m

# Load data here (Locations and Distances DataFrames)
Locations = pd.read_csv(f'{region}-locations.csv')

Distances = pd.read_csv(f'{region}-distance-matrix.csv', index_col='name')

# Set parameter for triangle_pruning
t_value = 1000  # Adjust this threshold value as needed

# Visualise triangle_pruning results
m_triangle_pruned = visualise_triangle_pruned_routes(Locations, Distances, region, t_value)

# Display map
m_triangle_pruned

#### The code below Implements KNN routing

In [ ]:
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters


# Dictionary to store computed distances
distance_cache = {}

def knn_route(x, df, k):
    idx1 = x.name  # Use index as the location name
    distances = []
    lat1 = df.loc[idx1, 'latitude']
    lon1 = df.loc[idx1, 'longitude']
    for idx2 in df.index:
        lat2 = df.loc[idx2, 'latitude']
        lon2 = df.loc[idx2, 'longitude']

        # Check if distance is already calculated
        if (idx1, idx2) in distance_cache:
            distance = distance_cache[(idx1, idx2)]
        elif (idx2, idx1) in distance_cache:
            distance = distance_cache[(idx2, idx1)]
        else:
            distance = calculate_distance((lat1, lon1), (lat2, lon2))
            distance_cache[(idx1, idx2)] = distance  # Store the calculated distance

        distances.append(distance)
    
    nearest_neighbors = sorted(enumerate(distances), key=lambda x: x[1])[1:k+1]  # Exclude the location itself
    return [distances[idx] for idx, _ in nearest_neighbors]


def visualise_knn_routes(locations_df, k):
    # Create a map centered around the first location
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Apply knn_route function to get the k-nearest neighbors distances for each location
    knn_pruned_distances = locations_df.apply(knn_route, df=locations_df, k=k, axis=1)

    # Create routes DataFrame from k-NN route results
    routes_data = []
    for idx1, distances in knn_pruned_distances.items():
        nearest_neighbors = [idx2 for idx2, distance in enumerate(distances) if distance > 0]
        for idx2 in nearest_neighbors:
            routes_data.append({'idx1': idx1, 'idx2': idx2})
    routes_df = pd.DataFrame(routes_data)
    
    # Iterate through the routes_df DataFrame and update the geometry column
    route_coordinates = []
    for i, row in routes_df.iterrows():
        idx1 = row['idx1']  # Use the index as location name
        idx2 = row['idx2']  # Use the index as location name

        origin = (locations_df.loc[idx1, 'latitude'], locations_df.loc[idx1, 'longitude'])
        destination = (locations_df.loc[idx2, 'latitude'], locations_df.loc[idx2, 'longitude'])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {idx1} and {idx2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
            route_coordinates.append(coordinates)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for {idx1} to {idx2}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{name1} to {name2}"
        )
        polyline_obj.add_to(m)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        if row['location_type'] == 'camp':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='green',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        elif row['location_type'] == 'town':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='darkorange',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        else:
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        
    # Save the route_coordinates to the CSV file
    csv_file = f"{region}-knn-pruned-route-coords.csv"
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["latitude", "longitude"])
        for coordinates in route_coordinates:
            writer.writerows(coordinates)

    print(f"Coordinates saved as {csv_file}")
    
    # Save the map as an HTML file
    map_file = f'images/{region}-knn-pruned-route-map.html'
    m.save(map_file)
    print(f"Map saved as {map_file}")
    
    # Create a CSV file to store location pairs and their distances
    csv_file = f"{region}-knn-distances.csv"
    
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["location1", "location2", "distance"])  # Write header

        # Apply knn_route function to get the k-nearest neighbors distances for each location
        knn_pruned_distances = locations_df.apply(knn_route, df=locations_df, k=k, axis=1)

        # Write distances to CSV file
        for idx1, distances in knn_pruned_distances.items():
            location1 = locations_df.loc[idx1, 'name']
            nearest_neighbors = [(idx2, distance) for idx2, distance in enumerate(distances) if distance > 0]
            for idx2, distance in nearest_neighbors:
                location2 = locations_df.loc[idx2, 'name']
                
                # Skip if location1 and location2 are the same
                if location1 == location2:
                    continue
                
                # Round distance to two decimal places
                distance = round(distance, 2)
                
                writer.writerow([location1, location2, distance])  # location1, location2, distance

    print(f"Distances saved as {csv_file}")

    # Display the map
    return m

# Load data here (Locations DataFrame)
Locations = pd.read_csv(f'{region}-locations.csv')

# Set parameter for k-NN route
k_value = 3  # Adjust this value for the number of nearest neighbors

# Visualise k-NN route results
m_knn_route = visualise_knn_routes(Locations, k_value)

# Display map
m_knn_route


#### Implementing Sequential Pruning chining Triangle Pruning, Geo Pruning and KNN Pruning in sequence

In [ ]:
def calculate_distance(coord1, coord2):
    return geodesic(coord1, coord2).meters


def get_polyline_string(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/driving/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200 and data.get("code") == "Ok":
        route = data.get("routes")[0]
        polyline_string = route.get("geometry")
        return polyline_string
    else:
        return None
    

def visualise_routes(locations_df, routes_df, region):
    route_coordinates = []
    start_lat = locations_df.loc[0, 'latitude']
    start_lon = locations_df.loc[0, 'longitude']
    m = folium.Map(location=[start_lat, start_lon], zoom_start=6)

    # Iterate through the routes_df DataFrame and update the geometry column
    for i, row in routes_df.iterrows():
        idx1 = row['name1']  # Use the index as location name
        idx2 = row['name2']  # Use the index as location name

        origin = (locations_df.loc[idx1, 'latitude'], locations_df.loc[idx1, 'longitude'])
        destination = (locations_df.loc[idx2, 'latitude'], locations_df.loc[idx2, 'longitude'])
        geometry_str = get_polyline_string(origin, destination)
        routes_df.at[i, 'geometry'] = geometry_str

        # Skip rows with NaN 'geometry' values
        if pd.isna(geometry_str):
            print(f"Warning: No route between {idx1} and {idx2}, skipping.")
            continue

        # Attempt to decode the polyline
        try:
            coordinates = polyline.decode(geometry_str)
            route_coordinates.append(coordinates)
        except ValueError:
            print(f"Warning: Invalid 'geometry' value for {idx1} to {idx2}, skipping.")
            continue

        # Continue with existing code
        polyline_obj = folium.PolyLine(
            locations=coordinates,
            color='blue',
            weight=2,
            opacity=0.9,
            popup=f"{idx1} to {idx2}"
        )
        polyline_obj.add_to(m)

    # Add markers for each location
    for i, row in locations_df.iterrows():
        name = row['name']
        lat = row['latitude']
        lon = row['longitude']
        if row['location_type'] == 'camp':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='green',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        elif row['location_type'] == 'town':
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='darkorange',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        else:
            folium.CircleMarker(
                location=(lat, lon),
                radius=6,
                color='red',
                fill=True,
                fill_color='blue',
                popup=name
            ).add_to(m)
        
    # Save the route_coordinates to the CSV file
    csv_file = f"{region}-pruned-route-coords.csv"
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["latitude", "longitude"])
        for coordinates in route_coordinates:
            writer.writerows(coordinates)

    print(f"Coordinates saved as {csv_file}")

    # Save the map as an HTML file
    map_file = f'images/{region}-pruned-route-map.html'
    m.save(map_file)
    print(f"Map saved as {map_file}")
    
    # Create a CSV file to store location pairs and their pruned distances
    csv_file = f"{region}-chained-pruned-distances.csv"

    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["location1", "location2", "distance"])  # Write header

        # Iterate through the geo_pruned_distances_df DataFrame
        for idx1, distances in geo_pruned_distances_df.iterrows():
            location1 = Locations.loc[idx1, 'name']
            nearest_neighbors = [(idx2, distance) for idx2, distance in enumerate(distances) if distance > 0]
            for idx2, distance in nearest_neighbors:
                location2 = Locations.loc[idx2, 'name']

                # Skip if location1 and location2 are the same
                if location1 == location2:
                    continue

                # Round distance to two decimal places
                distance = round(distance, 2)

                writer.writerow([location1, location2, distance])  # location1, location2, distance

    print(f"Pruned distances saved as {csv_file}")

    # Display the map
    return m


def triangle_pruning(x, df, num_neighbors):
    name = x.name
    distances = []
    # Extract latitude and longitude of the current location
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    # Calculate distances to all other locations
    for idx, row in df.iterrows():
        if row['name'] != name:
            x2 = row['latitude']
            y2 = row['longitude']
            distance = geodesic((x1, y1), (x2, y2)).meters
            distances.append((idx, distance))
    
    # Sort distances and select the nearest neighbors
    nearest_neighbors = sorted(distances, key=lambda x: x[1])[:num_neighbors]
    return [idx for idx, _ in nearest_neighbors]


def geo_pruning(x, df, k, b):
    name = x.name
    distances = []
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    for xx in df['name']:
        x2 = df.loc[df['name'] == xx, 'latitude'].values[0]
        y2 = df.loc[df['name'] == xx, 'longitude'].values[0]
        distance = geodesic((x1, y1), (x2, y2)).meters
        distances.append(distance)
    knn_index = sorted(range(len(distances)), key=lambda i: distances[i])[k]
    knn = distances[knn_index]
    distances = [d if d <= knn * b else 0 for d in distances]
    return distances


def knn_route(x, df, k):
    name = x.name
    distances = []
    x1 = df.loc[df['name'] == name, 'latitude'].values[0]
    y1 = df.loc[df['name'] == name, 'longitude'].values[0]
    for xx in df['name']:
        x2 = df.loc[df['name'] == xx, 'latitude'].values[0]
        y2 = df.loc[df['name'] == xx, 'longitude'].values[0]
        distance = geodesic((x1, y1), (x2, y2)).meters
        distances.append(distance)
    nearest_neighbors = sorted(range(len(distances)), key=lambda i: distances[i])[1:k+1]
    return [distances[idx] for idx in nearest_neighbors]


# Load data here (Locations and Distances DataFrames)
Locations = pd.read_csv(f'{region}-locations.csv')

Distances = pd.read_csv(f'{region}-distance-matrix.csv', index_col='name')

# Set parameters for pruning and k-NN route
t_value = 1000  # Adjust this value for triangle pruning
k_value = 3  # Adjust this value for the number of nearest neighbors
k_distance = 1000  # Adjust this value for k-NN route

# Triangle Pruning
pruned_distances_df = Distances.apply(triangle_pruning, args=(Locations, t_value,))

# Geo Pruning
geo_pruned_distances_df = pruned_distances_df.apply(geo_pruning, args=(Locations, k_value, 1.1))

# k-NN Route
knn_routes_df = geo_pruned_distances_df.apply(knn_route, args=(Locations, k_value))

# Create routes DataFrame from k-NN Route results
routes_data = []
for name, row in knn_routes_df.iterrows():
    nearest_neighbors = [col for col, distance in zip(Distances.index, row) if distance > 0]
    for neighbor in nearest_neighbors:
        routes_data.append({'name1': name, 'name2': neighbor})
routes_df = pd.DataFrame(routes_data)

# Visualise routes
m_routes = visualise_routes(Locations, routes_df, region)

# Display map
m_routes

Experiment with different pruning algorithm and observe the results.